In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
import sqlalchemy
import sys, os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from config import pw

In [4]:
df = pd.read_csv("../data/CleanedCSV/globalWineCleaned.csv")
df

,Unnamed: 0,vintage,country,county,designation,points,price,province,title,variety,winery
0,0,1919,Spain,Cava,1919 Brut Selecció,88,13.0,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,1,1929,Italy,Vernaccia di San Gimignano,NaN,87,14.0,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,2,1929,Italy,Sangiovese di Romagna Superiore,Prugneto,84,15.0,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,3,1934,Portugal,NaN,Reserva Velho,93,495.0,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,4,1945,France,Rivesaltes,Legend Vintage,95,350.0,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand
...,...,...,...,...,...,...,...,...,...,...,...
24566,24992,2016,Austria,NaN,Lust & Laune,88,15.0,Niederösterreich,Familie Zull 2016 Lust & Laune Rosé (Niederöst...,Rosé,Familie Zull
24567,24993,2016,Austria,NaN,Vorder Seiber Federspiel,90,25.0,Wachau,Jäger 2016 Vorder Seiber Federspiel Grüner Vel...,Grüner Veltliner,Jäger
24568,24994,1821,Italy,Prosecco,Cuvèe 1821 Brut,86,15.0,Veneto,Zonin NV Cuvèe 1821 Brut (Prosecco),Glera,Zonin
24569,24995,1827,Spain,Jerez,Pedro Ximenez 1827 Sweet Sherry,94,14.0,Andalucia,Osborne NV Pedro Ximenez 1827 Sweet Sherry She...,Sherry,Osborne


In [5]:
cols = df.select_dtypes(include=[np.object]).columns
df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df

C:\Users\katty\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


,Unnamed: 0,vintage,country,county,designation,points,price,province,title,variety,winery
0,0,1919,Spain,Cava,1919 Brut Seleccio,88,13.0,Catalonia,L'Arboc NV 1919 Brut Seleccio Sparkling (Cava),Sparkling Blend,L'Arboc
1,1,1929,Italy,Vernaccia di San Gimignano,NaN,87,14.0,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,2,1929,Italy,Sangiovese di Romagna Superiore,Prugneto,84,15.0,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,3,1934,Portugal,NaN,Reserva Velho,93,495.0,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,4,1945,France,Rivesaltes,Legend Vintage,95,350.0,Languedoc-Roussillon,Gerard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gerard Bertrand
...,...,...,...,...,...,...,...,...,...,...,...
24566,24992,2016,Austria,NaN,Lust & Laune,88,15.0,Niederosterreich,Familie Zull 2016 Lust & Laune Rose (Niederost...,Rose,Familie Zull
24567,24993,2016,Austria,NaN,Vorder Seiber Federspiel,90,25.0,Wachau,Jager 2016 Vorder Seiber Federspiel Gruner Vel...,Gruner Veltliner,Jager
24568,24994,1821,Italy,Prosecco,Cuvee 1821 Brut,86,15.0,Veneto,Zonin NV Cuvee 1821 Brut (Prosecco),Glera,Zonin
24569,24995,1827,Spain,Jerez,Pedro Ximenez 1827 Sweet Sherry,94,14.0,Andalucia,Osborne NV Pedro Ximenez 1827 Sweet Sherry She...,Sherry,Osborne


In [6]:
df

,Unnamed: 0,vintage,country,county,designation,points,price,province,title,variety,winery
0,0,1919,Spain,Cava,1919 Brut Seleccio,88,13.0,Catalonia,L'Arboc NV 1919 Brut Seleccio Sparkling (Cava),Sparkling Blend,L'Arboc
1,1,1929,Italy,Vernaccia di San Gimignano,NaN,87,14.0,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,2,1929,Italy,Sangiovese di Romagna Superiore,Prugneto,84,15.0,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,3,1934,Portugal,NaN,Reserva Velho,93,495.0,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,4,1945,France,Rivesaltes,Legend Vintage,95,350.0,Languedoc-Roussillon,Gerard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gerard Bertrand
...,...,...,...,...,...,...,...,...,...,...,...
24566,24992,2016,Austria,NaN,Lust & Laune,88,15.0,Niederosterreich,Familie Zull 2016 Lust & Laune Rose (Niederost...,Rose,Familie Zull
24567,24993,2016,Austria,NaN,Vorder Seiber Federspiel,90,25.0,Wachau,Jager 2016 Vorder Seiber Federspiel Gruner Vel...,Gruner Veltliner,Jager
24568,24994,1821,Italy,Prosecco,Cuvee 1821 Brut,86,15.0,Veneto,Zonin NV Cuvee 1821 Brut (Prosecco),Glera,Zonin
24569,24995,1827,Spain,Jerez,Pedro Ximenez 1827 Sweet Sherry,94,14.0,Andalucia,Osborne NV Pedro Ximenez 1827 Sweet Sherry She...,Sherry,Osborne


In [7]:
# saving the DataFrame as a CSV file 
winesCSV = df.to_csv('../data/CleanedCSV/winesCleaned.csv', index = True)